# 5. Język interpertowany AWK - część 1

Kolejnym, bardzo potężnym narzędziem do przetwarzania plików tekstowych w systemach UNIX-owych jest program awk. Może być traktowany jako samodzielny skryptowy język programowania. Może być używany zarówno jako samodzielna komenda w terminalu, a także używane w skryptach oraz sam w sobie może się składać z serii poleceń tworzących jeden skrypt awk. Zawiera bardzo wiele pożytecznych struktur danych, m.in. są to: tablice asocjacyjne (czyli takie, które indeksuje sie nie za pomocą liczb, ale słów), zmienne znakowe (string) oraz wyrażenia regularne. Tutaj skupimy się na wykorzystaniu języka `awk` w postaci jednoliniowych komend z użyciem wcześniej wymienionych struktur.

## 5.1 Struktura `awk` i użycie postawowych komend.

Składnia wywołania:
```bash
awk '<warunek> { <polecenie> }' <plik_tekstowy>
```

Po komendzie `awk`, polecenia które program będzie wykonywał należy "opakować" cudzysłowem `''`, a na trzeba podać nazwę pliku do przetworzenia. Komendę `awk` można również użyć w potoku `|`. 

W `awk` wbudowana jest komenda `print` służąca do wypisywania zawartości pliku. Każda linia jest rozdzielana na poszczególne pola ze względu na domyślny separator (spacje, tabulator).

Przykłady:

>Wypisuje tylko pierwszą kolumnę z pliku, jeśli wartość jest większa od 999

In [24]:
awk '$1 > 999 {print $1}' equations.dat 

999.29
999.33
999.39
999.14
999.61
999.96
999.71
999.25
999.72


> Bez podania polecenia, warunek zadziała na całą linię (zostanie ona wypisana)

In [25]:
awk '$1 > 999' equations.dat 

999.29 - 518.28
999.33 / 231.7
999.39 + 819.7
999.14 / 889.47
999.61 - 463.58
999.96 + 734.22
999.71 / 183.59
999.25 - 915.72
999.72 - 986.44


> Analogicznie, bez podania warunku, polecenie zadziała na wszystkie linie w pliku. Dla czytelności, wyświetlę tylko 10 ostatnich linii pliku `equations.dat`. Dodatkowo połączymy efekt działania komendy `tail` z `awk` za pomocą potoku (pipe) `|`.

In [27]:
tail -n 10 equations.dat | awk '{print $2}'

+
/
+
/
/
-
/
-
+
+


## 5.2 Wbudowane zmienne i opreacje matematyczne.

W `awk` można również używać wbudowanych zmiennych, są to m. in.:
- umożliwiające dostęp do poszczególnych pól: `$1`, `$2`, `$3`
- `$0`: odnosi się do całej linii
- `NR`: odnosi się do numeru linii w pliku
- `NF`: odnosi się do ilości pól w linii, do ostatniego pola można się odnieść poprzez `$NF`
- `FILENAME`: zawiera nazwę obecnie przetwarzanego pliku
- `FS`: zmienna zawierająca obecny separator pola, można tę zmienną nadpisać aby zmienić separator
- `RS`: zmienna zawierająca obecny seperator rekordu, domyślnie jest to znak nowej linii
- `OFS`: zmienna zawierająca separator używany przy wypisywaniu pól, domyślnie jest to "spacja"
- `ORS`: zmienna zawierająca separator używany przy wypisywaniu rekordów, domyślnie jest to "znak nowej linii"

Inne operacje:
- `BEGIN` : nie jest to zmienna, ale warunek który powoduje wykonanie polecenia na początku czytania pliku (dzięki temu można się odnieść do pierwszej linii)
- `END` : nie jest to zmienna, ale warunek który powoduje wykonanie polecenia na końcu czytania pliku (dzięki temu można się odnieść do ostatniej linii) 

Dodatkowo można też wykonywać operacje matematyczne na zmiennych (oczywiście jeśli są to liczby), za pomocą:<br>`+ : dodawanie`, `- : odejmowanie`, `* : mnożenie`, `/ : dzielenie`, `% : modulo`, `^ : potęgowanie`.<br>Wyniki działań można również przypisywać do zmiennych zdefiniowanych przez użytkownika. Poniżej przykłady:

> W tym przykładzie podzielone zostały liczby z drugiej linii pliku `equations.dat`

In [45]:
awk 'NR==2 {print "liczby: "$1,$3,"| wynik dzielenia: ",$1/$3}' equations.dat

liczby: 782.65 848.9 | wynik dzielenia:  0.921958


In [62]:
awk 'NR==85 {print "liczby: "$1,$3,"| wynik potęgowania: ",$1^2}' equations.dat

liczby: 651.11 573.97 | wynik potęgowania:  423944


> Za pomocą `awk` można też w łatwy sposób napisać skrypt liczący bazę czasową z pierwszego notebook-a.

In [65]:
head TIC102090493-s2-121s_lc3_fppt.dat

# TBJD-2450000      FLUX           FLUX_ERR
  1354.1131991     -3.5355597      0.6838813
  1354.1145881     -3.0284873      0.6841874
  1354.1173659      0.7362481      0.6849412
  1354.1187548      1.0039511      0.6853946
  1354.1201437      2.0536179      0.6856015
  1354.1215327      2.9236379      0.6860147
  1354.1229216      3.1528239      0.6861270
  1354.1243105      1.7814821      0.6857486
  1354.1256994      2.6458713      0.6859725


In [77]:
start=$(awk 'NR==2{print $1}' TIC102090493-s2-121s_lc3_fppt.dat)  # zapisuję pierwsze pole z drugiej linii

In [78]:
end=$(awk 'END{print $1}' TIC102090493-s2-121s_lc3_fppt.dat)    # zapisuję pierwsze pole z ostatniej linii

In [81]:
echo $end-$start | bc -l         # wyliczenie bazy czasowej

27.4044052


> Przykład jak zmienić separator pól: użyta zostanie __kropka__ `.` Chcę, aby na samym początku ustawiony został separator. Gwarantuje mi to `BEGIN {FS="."}`. Muszę następnie oddzielić średnikiem to polecenie od kolejnych. Potem wypisuję pierwsze pole, dla linii mniejszych od 10 i (drugi warunek dodaję za pomocą `&&`) większe od 1.

In [120]:
awk 'BEGIN{FS="."}; NR<10 && NR>1{print $1}' TIC102090493-s2-121s_lc3_fppt.dat

  1354
  1354
  1354
  1354
  1354
  1354
  1354
  1354


## 5.3 Wyrażenia regularne z `awk`

`awk` również używa wyrażeń regularnych (por. tabelka z notebook-a nr 4)

In [121]:
head equations.dat

724.96 * 1.37
782.65 - 848.9
805.51 + 981.97
296.5 / 132.77
477.15 / 217.23
149.96 - 418.28
707.83 / 241.16
750.53 - 329.8
401.87 / 319.88
717.33 - 189.74


In [137]:
awk '/[+-]/' equations.dat | awk 'NR<10'   # wypisze 10 pierwszych linii, które zawierają znaki + lub -

782.65 - 848.9
805.51 + 981.97
149.96 - 418.28
750.53 - 329.8
717.33 - 189.74
443.56 - 666.54
214.57 + 286.47
464.9 + 656.97
703.0 - 794.15


> Wypisze pierwszy element linii, które zawierają frazę która zaczyna wiersz od A lub a, po kótrej nastepuje dowolny znak i kończy się dowolnym znakiem z wyłączeniem spacji.

In [179]:
awk '/^[Aa].[^ ]/ {print $1}' bd_dr.txt

And
And
And
And
And
All
And
And
And
And
Across
And
And
And
All


<br>c.d.n.